In [12]:
import torch as pt
import torch.nn as nn
from torch.utils.data import random_split
from src.dataset import MolDataset
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd

In [3]:
device = "cuda" if pt.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
dataset = MolDataset("../data/preprocessed/X.pk", 
                     y_datafile="../data/preprocessed/Y.pk", normal = True)

In [70]:
X_original = pd.read_pickle("../data/preprocessed/X.pk")

In [16]:
y = pd.read_pickle("../data/preprocessed/Y.pk")

In [26]:
labels = dataset[:][1]
labels.sum(axis=0)

array([ 2041.,  2835., 65835.])

# Load 

In [119]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.hid = nn.Linear(hidden_size, hidden_size) 
        self.hid2 = nn.Linear(hidden_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.hid(out)
        out = self.relu(out)
        out = self.hid2(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [120]:
n_categories = 3
n_features = 42
learning_rate = 1e-5
n_hidden = 512
# n_layers = config_runtime['layers']


model = NeuralNet(n_features,n_hidden,n_categories).to(device)
model.load_state_dict(pt.load("../model/torch/model.pt"))
model.eval()

NeuralNet(
  (fc1): Linear(in_features=42, out_features=512, bias=True)
  (relu): ReLU()
  (hid): Linear(in_features=512, out_features=512, bias=True)
  (hid2): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)

# Test

In [121]:
ids = [8,2100,20000, 3674,9867]

In [76]:
scaler = joblib.load("../model/torch/scaler.save") 

In [78]:
X_mat = np.array(X_original.loc[:, X_original.columns != 'Id'].values,
                 dtype=np.float32)
X_mat = scaler.fit_transform(X_mat)

In [82]:
# X_mat[13] dataset[13]

In [101]:
print(X_mat[None, :].shape, dataset[13:59][0].shape)

(1, 70711, 42) (1, 46, 42)


In [122]:
X1 = dataset[ids][0]
# X1 = X_mat[None, ids,:]
treu_y = dataset[ids][1]
X = pt.tensor(X1, device=device)
# # X = torch.unsqueeze(X, dim=0)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(2)
# print(f"Predicted class: {y_pred}")

In [123]:
treu_y

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [124]:
pred_probab

tensor([[[0.3569, 0.3597, 0.1007],
         [0.1990, 0.1780, 0.1670],
         [0.1108, 0.0838, 0.3021],
         [0.2249, 0.2648, 0.1521],
         [0.1085, 0.1137, 0.2781]]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [125]:
pred_probab.argmax(2).cpu()

tensor([[1, 0, 2, 1, 2]])

# Real predictions


In [126]:
X_test = pd.read_pickle("../data/preprocessed/test.pk")

In [127]:
X_test_mat = np.array(X_test.loc[:, X_test.columns != 'Id'].values,
                 dtype=np.float32)
X_test_mat = scaler.fit_transform(X_test_mat)
X_id = list(X_test.loc[:,'Id'])

In [131]:
X = pt.tensor(X_test_mat[None, :,:], device=device)
# # X = torch.unsqueeze(X, dim=0)

logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)

In [136]:
X

tensor([[[-0.0708, -0.1973,  0.1632,  ...,  0.1958, -0.0197,  1.0612],
         [-0.0708, -0.1973,  6.2304,  ...,  2.6953, -0.7196,  1.9774],
         [ 8.4149, -0.1973,  2.4745,  ...,  0.6957,  1.3801, -1.6874],
         ...,
         [-0.0708, -0.1973,  1.3189,  ..., -0.3041,  0.6802, -0.3131],
         [-0.0708, -0.1973,  1.8967,  ...,  0.1958, -0.0197, -0.7712],
         [-0.0708, -0.1973, -0.7035,  ..., -0.3041, -0.7196,  0.6031]]],
       device='cuda:0')

In [133]:
pred_probab

tensor([[[2.5525e-05, 2.6870e-05, 3.3730e-05],
         [4.1668e-04, 6.2742e-05, 3.1555e-06],
         [2.6150e-04, 1.1765e-04, 5.8290e-06],
         ...,
         [1.0146e-04, 7.5468e-05, 1.0910e-05],
         [6.9153e-05, 6.0813e-05, 1.5180e-05],
         [3.8105e-05, 3.4889e-05, 2.8269e-05]]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [142]:
vals = np.array(pred_probab.argmax(2).cpu())[0]

In [152]:
np.sum(vals==0)

9329